In [1]:
import brunoflow as bf
from brunoflow.ad.utils import check_node_equals_tensor, check_node_allclose_tensor
from jax import numpy as jnp
import transformers
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForMaskedLM, 
    BertForMaskedLM, 
    BertTokenizer, 
    BertTokenizerFast, 
    BertEmbeddings,
    BfBertEmbeddings,
    BertConfig,
)

torch.manual_seed(0)

/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Establish data
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
text = ["hello I want to eat some [MASK] meat today. It's thanksgiving [MASK] all!", "yo yo what's up"]

# tokenize text and pass into model
tokens = tokenizer(text, return_tensors="pt", padding=True)
input_ids = tokens["input_ids"]
print(input_ids)

tensor([[  101,  7592,  1045,  2215,  2000,  4521,  2070,   103,  6240,  2651,
          1012,  2009,  1005,  1055, 15060,   103,  2035,   999,   102],
        [  101, 10930, 10930,  2054,  1005,  1055,  2039,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]])


In [3]:
# Create BfBertEmbeddings and BertEmbeddings
config = BertConfig.from_pretrained(pretrained_model_name_or_path="/home/kevin/code/rycolab/brunoflow/models/bert/config.json")
bf_embs = BfBertEmbeddings(config)
torch_embs = BertEmbeddings(config)
print(bf_embs)
print(torch_embs)

2022-12-22 18:07:11.873060: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


BfBertEmbeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1)
)
BertEmbeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


/home/kevin/code/rycolab/transformers/src/transformers/models/bert/modeling_bf_bert.py:175: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in zeros is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  self.register_buffer("token_type_ids", bf.Node(jnp.zeros(self.position_ids.shape, dtype=jnp.int64)), persistent=False) # todo is this 64 bit necessary?


In [4]:
# Save torch BertEmbeddings to file
save_path = "bertembeddings_torch.pt"
torch.save(torch_embs.state_dict(), save_path)

In [5]:
# Load torch BertEmbeddings into bf
bf_embs.load_state_dict(torch.load(save_path))

<All keys matched successfully>

In [6]:
bf_embs

BfBertEmbeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1)
)

In [7]:
assert(check_node_equals_tensor(bf_embs.word_embeddings.weight, torch_embs.word_embeddings.weight))
assert(check_node_equals_tensor(bf_embs.position_embeddings.weight, torch_embs.position_embeddings.weight))
assert(check_node_equals_tensor(bf_embs.token_type_embeddings.weight, torch_embs.token_type_embeddings.weight))
assert(check_node_equals_tensor(bf_embs.LayerNorm.weight, torch_embs.LayerNorm.weight))
# print(check_node_equals_tensor(bf_embs.dropout.weight, torch_embs.dropout.weight)) # this fails because dropout has no weights, I guess


In [12]:
# Compare output of forward pass of BfBertEmbeddings and BertEmbeddings on the text - they're equal!
jax_input_ids = jnp.array(input_ids.numpy(), dtype=int)
torch_embs.train(False)
out_bf = bf_embs(input_ids=jax_input_ids)
out_torch = torch_embs(input_ids=input_ids)
# print(out_bf.val)
# print(out_torch)
assert(check_node_allclose_tensor(out_bf, out_torch))

# Compare grad after backward pass
torch_embs.train(True)
out_torch.backward(gradient=torch.ones_like(out_torch))
# out_bf.backprop()

print("word_embeddings:", bf_embs.word_embeddings.weight.grad, torch_embs.word_embeddings.weight.grad)
print("position_embeddings:", bf_embs.position_embeddings.weight.grad, torch_embs.position_embeddings.weight.grad)
print("token_type_embeddings:", bf_embs.token_type_embeddings.weight.grad, torch_embs.token_type_embeddings.weight.grad)
print("LayerNorm:", bf_embs.LayerNorm.weight.grad, torch_embs.LayerNorm.weight.grad)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-4.7684e-07, -4.7684e-07, -4.7684e-07,  ..., -4.7684e-07,
         -4.7684e-07, -4.7684e-07],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [

In [9]:
# out_bf.backprop()